In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import psycopg2
import matplotlib.pyplot as plt
from datetime import datetime


# Database Connection
def connect_db():
    return psycopg2.connect(
        host="localhost",
        database="user",  # Database name
        user="postgres",
        password="root"
    )


# Setup Database
def setup_database():
    conn = connect_db()
    cursor = conn.cursor()

    # Create `users` table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS users (
            username VARCHAR(50) PRIMARY KEY,
            password VARCHAR(50) NOT NULL
        )
    """)

    # Create `screen_time` table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS screen_time (
            username VARCHAR(50),
            Date DATE NOT NULL,
            Screen_Time FLOAT,
            Phone_Unlocks INT,
            WhatsApp FLOAT,
            Instagram FLOAT,
            Facebook FLOAT,
            YouTube FLOAT,
            Chrome FLOAT,
            Games FLOAT,
            Others FLOAT,
            PRIMARY KEY (username, Date),
            FOREIGN KEY (username) REFERENCES users(username) ON DELETE CASCADE
        )
    """)

    conn.commit()
    conn.close()


# Signup Function
def signup():
    username = entry_username.get()
    password = entry_password.get()

    if not username or not password:
        messagebox.showerror("Error", "Please fill in all fields.")
        return

    try:
        conn = connect_db()
        cursor = conn.cursor()
        cursor.execute("INSERT INTO users (username, password) VALUES (%s, %s)", (username, password))
        conn.commit()
        conn.close()
        messagebox.showinfo("Success", "Signup successful! You can now log in.")
    except psycopg2.IntegrityError:
        conn.rollback()
        conn.close()
        messagebox.showerror("Error", "Username already exists!")
    except Exception as e:
        messagebox.showerror("Error", f"Database error: {e}")


# Login Function
def login():
    username = entry_username.get()
    password = entry_password.get()

    if not username or not password:
        messagebox.showerror("Error", "Please fill in all fields.")
        return

    try:
        conn = connect_db()
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM users WHERE username=%s AND password=%s", (username, password))
        user = cursor.fetchone()
        conn.close()

        if user:
            messagebox.showinfo("Success", "Login successful!")
            login_window.destroy()
            main_app(username)  # Pass the logged-in username to the main app
        else:
            messagebox.showerror("Error", "Invalid credentials!")
    except Exception as e:
        messagebox.showerror("Error", f"Database error: {e}")


# Save Screen Time Data
def save_screen_time(username, entry_date, entry_screen_time, entry_phone_unlocks,
                     entry_whatsapp, entry_instagram, entry_facebook,
                     entry_youtube, entry_chrome, entry_games, entry_others):
    try:
        date = datetime.strptime(entry_date.get(), "%Y-%m-%d").date()
        screen_time = float(entry_screen_time.get())
        phone_unlocks = int(entry_phone_unlocks.get())
        whatsapp = float(entry_whatsapp.get())
        instagram = float(entry_instagram.get())
        facebook = float(entry_facebook.get())
        youtube = float(entry_youtube.get())
        chrome = float(entry_chrome.get())
        games = float(entry_games.get())
        others = float(entry_others.get())

        conn = connect_db()
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO screen_time 
            (username, Date, Screen_Time, Phone_Unlocks, WhatsApp, Instagram, Facebook, YouTube, Chrome, Games, Others)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (username, Date) DO UPDATE SET
            Screen_Time = EXCLUDED.Screen_Time,
            Phone_Unlocks = EXCLUDED.Phone_Unlocks,
            WhatsApp = EXCLUDED.WhatsApp,
            Instagram = EXCLUDED.Instagram,
            Facebook = EXCLUDED.Facebook,
            YouTube = EXCLUDED.YouTube,
            Chrome = EXCLUDED.Chrome,
            Games = EXCLUDED.Games,
            Others = EXCLUDED.Others
        """, (username, date, screen_time, phone_unlocks, whatsapp, instagram, facebook, youtube, chrome, games, others))
        conn.commit()
        conn.close()

        messagebox.showinfo("Success", "Data saved successfully!")
        load_data(username)
    except Exception as e:
        messagebox.showerror("Error", f"Failed to save data: {e}")



# Load Screen Time Data
def load_data(username):
    for item in tree.get_children():
        tree.delete(item)

    try:
        conn = connect_db()
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM screen_time WHERE username=%s ORDER BY Date DESC", (username,))
        rows = cursor.fetchall()
        conn.close()

        for row in rows:
            tree.insert("", tk.END, values=row[1:])  # Skip the username column
    except Exception as e:
        messagebox.showerror("Error", f"Failed to load data: {e}")


# Analyze Data
def analyze_data(username):
    try:
        conn = connect_db()
        cursor = conn.cursor()
        cursor.execute("SELECT Date, Screen_Time FROM screen_time WHERE username=%s ORDER BY Date", (username,))
        data = cursor.fetchall()
        conn.close()

        if not data:
            messagebox.showinfo("No Data", "No data available for analysis.")
            return

        # Prepare data for plotting
        dates = [row[0] for row in data]
        screen_time = [row[1] for row in data]

        # Plotting
        plt.figure(figsize=(10, 6))
        plt.plot(dates, screen_time, marker="o", linestyle="-", color="b", label="Screen Time (hrs)")
        plt.title("Daily Screen Time Analysis")
        plt.xlabel("Date")
        plt.ylabel("Screen Time (hrs)")
        plt.grid(True)
        plt.legend()
        plt.show()
    except Exception as e:
        messagebox.showerror("Error", f"Failed to analyze data: {e}")


# Main Application
def main_app(username):
    root = tk.Tk()
    root.title("Screen Time Tracker")
    root.geometry("950x700")
    root.configure(bg="lightgray")

    tk.Label(root, text=f"Welcome, {username}!", font=("Arial", 16, "bold"), bg="lightgray", fg="blue").pack(pady=10)

    # Form
    frame_form = tk.Frame(root, bg="lightgray")
    frame_form.pack(pady=10)

    labels = [
        "Date (YYYY-MM-DD):", "Screen Time (hrs):", "Phone Unlocks:",
        "WhatsApp Usage:", "Instagram Usage:", "Facebook Usage:",
        "YouTube Usage:", "Chrome Usage:", "Games Usage:", "Others Usage:"
    ]

    entries = []

    for i, label_text in enumerate(labels):
        tk.Label(frame_form, text=label_text, font=("Arial", 12), bg="lightgray").grid(row=i, column=0, sticky="w", padx=10, pady=5)
        entry = tk.Entry(frame_form, width=30)
        entry.grid(row=i, column=1, padx=10, pady=5)
        entries.append(entry)

    (entry_date, entry_screen_time, entry_phone_unlocks,
     entry_whatsapp, entry_instagram, entry_facebook,
     entry_youtube, entry_chrome, entry_games, entry_others) = entries

    # Buttons
    frame_buttons = tk.Frame(root, bg="lightgray")
    frame_buttons.pack(pady=10)

    tk.Button(frame_buttons, text="Save Data", command=lambda: save_screen_time(username), bg="green", fg="white", font=("Arial", 12)).pack(side="left", padx=10)
    tk.Button(frame_buttons, text="Analyze Data", command=lambda: analyze_data(username), bg="orange", fg="white", font=("Arial", 12)).pack(side="left", padx=10)

    # Data Table
    frame_table = tk.Frame(root, bg="lightgray")
    frame_table.pack(pady=10)

    columns = ["Date", "Screen Time", "Phone Unlocks", "WhatsApp", "Instagram", "Facebook", "YouTube", "Chrome", "Games", "Others"]
    global tree
    tree = ttk.Treeview(frame_table, columns=columns, show="headings", height=15)

    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=90)

    tree.pack(pady=20)

    load_data(username)
    root.mainloop()


# Login/Signup GUI
def login_signup():
    global login_window, entry_username, entry_password

    login_window = tk.Tk()
    login_window.title("Login / Signup")
    login_window.geometry("400x300")
    login_window.configure(bg="lightgray")

    tk.Label(login_window, text="Login / Signup", font=("Arial", 16, "bold"), bg="lightgray", fg="blue").pack(pady=10)

    tk.Label(login_window, text="Username:", font=("Arial", 12), bg="lightgray").pack(pady=5)
    entry_username = tk.Entry(login_window, width=30)
    entry_username.pack(pady=5)

    tk.Label(login_window, text="Password:", font=("Arial", 12), bg="lightgray").pack(pady=5)
    entry_password = tk.Entry(login_window, show="*", width=30)
    entry_password.pack(pady=5)

    tk.Button(login_window, text="Login", command=login, bg="green", fg="white", font=("Arial", 12)).pack(pady=10)
    tk.Button(login_window, text="Signup", command=signup, bg="orange", fg="white", font=("Arial", 12)).pack(pady=5)

    login_window.mainloop()


# Setup Database and Run Application
setup_database()  # Ensure the database is ready
login_signup()  # Launch the login/signup interface
